In [ ]:
%matplotlib inline
import json
import numpy as np
import pandas as pd
import plotly.offline as ply
#import rapidjson
#import ujson
import plotly.graph_objs as go
import datetime
from plotly import tools
from time import time, mktime
ply.init_notebook_mode(connected=True)

In [ ]:
start = time()
#data = json.load(open('arbitrage-logger-export.json'))['log_btc_usd']
data = json.load(open('log_today.json'))['log_btc_usd']
print('Execution time: {:.3f}'.format(time() - start))

In [ ]:
'''
start = time()
udata = ujson.load(open('arbitrage-logger-export.json'))
print('Execution time: {:.3f}'.format(time() - start))

start = time()
rdata = rapidjson.load(open('arbitrage-logger-export.json'))
print('Execution time: {:.3f}'.format(time() - start))
''';

In [ ]:
'''
start = time()


exchanges = {'bitstamp', 'cex', 'exmo', 'gdax'}
data_dict = {'timestamp': []}
for exch in exchanges:
    data_dict['bids_' + exch] = []
    data_dict['asks_' + exch] = []

for key, val in data.items():
    data_dict['timestamp'].append(key)
    for x in val['ticker']:
        exch = x['exchange']
        data_dict['bids_' + exch].append(float(x['bid']))
        data_dict['asks_' + exch].append(float(x['ask']))
df = pd.DataFrame(data_dict)


print('Execution time: {:.3f}'.format(time() - start))
''';

In [ ]:
start = time()


exchanges = {'bitstamp', 'cex', 'exmo', 'gdax'}
bids = {exch: 'bid_' + exch for exch in exchanges}
asks = {exch: 'ask_' + exch for exch in exchanges}
data_dict = {'timestamp': []}
for exch in exchanges:
    data_dict['bid_' + exch] = []
    data_dict['ask_' + exch] = []

for key, val in data.items():
    data_dict['timestamp'].append(key)
    for x in val['ticker']:
        exch = x['exchange']
        data_dict[bids[exch]].append(float(x['bid']))
        data_dict[asks[exch]].append(float(x['ask']))

df = pd.DataFrame(data_dict)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')


print('Execution time: {:.3f}'.format(time() - start))

In [ ]:
df.head()

In [ ]:
df['bid_max_exch'] = df[['bid_' + exch for exch in exchanges]].idxmax(axis = 1)
df['bid_max_val'] = df[['bid_' + exch for exch in exchanges]].max(axis = 1)
df['ask_min_exch'] = df[['ask_' + exch for exch in exchanges]].idxmin(axis = 1)
df['ask_min_val'] = df[['ask_' + exch for exch in exchanges]].min(axis = 1)
df.head()

In [ ]:
df.info()

In [ ]:
df.dropna(axis=0, how='any', inplace=True)  # drop rows with NA values
df = df[(df.T != 0).all()]                  # drop rows with zeroes (there is no data for corresponding exchanges)
df.info()

In [ ]:
df['avg'] = (df['bid_cex'] + df['bid_exmo'] + df['bid_bitstamp'] + df['bid_gdax'] + df['ask_cex'] + df['ask_exmo'] + df['ask_bitstamp'] + df['ask_gdax']) / 8

In [ ]:
df.describe()

In [ ]:
'''
start = time()


fig = tools.make_subplots(rows=4, cols=2, print_grid=False)
i = 1
for exch in exchanges:
    name = 'bid_' + exch
    fig.append_trace(go.Histogram(x=df[name], name=name), i, 1)
    name = 'ask_' + exch
    fig.append_trace(go.Histogram(x=df[name], name=name), i, 2)
    i += 1
ply.iplot(fig)


print('Execution time: {:.3f}'.format(time() - start))
'''


start = time()

'''
fig = tools.make_subplots(rows=4, cols=1, print_grid=False)
i = 1
for exch in exchanges:
    name = exch
    fig.append_trace(go.Histogram(x = df['ask_' + exch] - df['bid_' + exch], name = name), i, 1)
    i += 1
ply.iplot(fig)
'''

for exch in exchanges:
    trace = go.Histogram(x = df['ask_' + exch] - df['bid_' + exch], name = exch)
    layout = go.Layout(title = exch)
    ply.iplot({'data': [trace], 'layout': layout})


print('Execution time: {:.3f}'.format(time() - start))

In [ ]:
start = time()

'''
fig = tools.make_subplots(rows=4, cols=1, print_grid=False)
i = 1
for exch in exchanges:
    name = exch
    fig.append_trace(go.Scatter(x = df['timestamp'], y = df['ask_' + exch] - df['bid_' + exch], name = name), i, 1)
    i += 1
ply.iplot(fig)
'''

for exch in exchanges:
    trace = go.Scatter(x = df['timestamp'], y = df['ask_' + exch] - df['bid_' + exch], name = exch)
    layout = go.Layout(title = exch)
    ply.iplot({'data': [trace], 'layout': layout})

print('Execution time: {:.3f}'.format(time() - start))

In [ ]:
start = time()

colors = {
    'bitstamp': {'ask': 'rgba(255, 0, 0, 0.8)', 'bid': 'rgba(255, 0, 0, 0.4)'},
    'cex': {'ask': 'rgba(0, 255, 0, 0.8)', 'bid': 'rgba(0, 255, 0, 0.4)'},
    'exmo': {'ask': 'rgba(0, 0, 255, 0.8)', 'bid': 'rgba(0, 0, 255, 0.4)'},
    'gdax': {'ask': 'rgba(0, 0, 0, 0.8)', 'bid': 'rgba(255, 224, 229, 0.4)'},
}

traces = []
for exch in exchanges:
    name = 'bid_' + exch
    marker = dict(color = colors[exch]['bid'])
    traces.append(go.Scatter(x=df['timestamp'], y=df[name], name=name, marker = marker))
    
    name = 'ask_' + exch
    marker = dict(color = colors[exch]['ask'])
    traces.append(go.Scatter(x=df['timestamp'], y=df[name], name=name, marker = marker))
ply.iplot(traces)


print('Execution time: {:.3f}'.format(time() - start))

In [ ]:
df2 = df[df['ask_exmo'] - df['bid_exmo'] <= 100]    # drop rows where Exmo behaves strangely
df2.info()

In [ ]:
start = time()

traces = []
colors = {
    'bitstamp': {'ask': 'rgba(255, 0, 0, 0.8)', 'bid': 'rgba(255, 0, 0, 0.3)'},
    'cex': {'ask': 'rgba(0, 255, 0, 0.8)', 'bid': 'rgba(0, 255, 0, 0.3)'},
    'exmo': {'ask': 'rgba(0, 0, 255, 0.8)', 'bid': 'rgba(0, 0, 255, 0.3)'},
    'gdax': {'ask': 'rgba(0, 0, 0, 0.8)', 'bid': 'rgba(0, 0, 0, 0.3)'},
}


for exch in exchanges:
    trace = go.Scatter(x = df2['timestamp'],
                       y = df2['bid_' + exch] - df2['avg'],
                       name = 'bid_' + exch,
                       marker = dict(color = colors[exch]['bid'])
                      )
    traces.append(trace)
    trace = go.Scatter(x = df2['timestamp'],
                       y = df2['ask_' + exch] - df2['avg'],
                       name = 'ask_' + exch,
                       marker = dict(color = colors[exch]['ask'])
                      )
    traces.append(trace)

    
layout = dict(
    title = 'Asks and Bids (centered)',
    yaxis = {'title': 'Price, USD'}
)
fig = dict(data=traces, layout=layout)
ply.iplot(fig, filename='test')


print('time: {:.3f}'.format(time() - start))

In [ ]:
start = time()


colors = {
    'bitstamp': {'ask': 'rgba(255, 0, 0, 0.8)', 'bid': 'rgba(255, 0, 0, 0.3)'},
    'cex': {'ask': 'rgba(0, 255, 0, 0.8)', 'bid': 'rgba(0, 255, 0, 0.3)'},
    'exmo': {'ask': 'rgba(0, 0, 255, 0.8)', 'bid': 'rgba(0, 0, 255, 0.3)'},
    'gdax': {'ask': 'rgba(0, 0, 0, 0.8)', 'bid': 'rgba(0, 0, 0, 0.3)'},
}


traces = []
for exch in exchanges:  
    plot_data = df2[df2['ask_min_exch'] == 'ask_' + exch]
    trace = go.Scatter(x = plot_data['timestamp'],
                       y = plot_data['ask_min_val'],
                       name = 'ask_' + exch,
                       mode = 'markers',
                       marker = dict(size = 3, color = colors[exch]['ask'])
                      )
    traces.append(trace)
    
    plot_data = df2[df2['bid_max_exch'] == 'bid_' + exch]
    trace = go.Scatter(x = plot_data['timestamp'],
                       y = plot_data['bid_max_val'],
                       name = 'bid_' + exch,
                       mode = 'markers',
                       marker = dict(size = 3, color = colors[exch]['bid'])
                      )
    traces.append(trace)

layout = dict(
    title = 'Max bids & Min asks',
    yaxis = {'title': 'Price, USD'}
)
fig = dict(data=traces, layout=layout)
ply.iplot(fig, filename='test')


print('time: {:.3f}'.format(time() - start))

In [ ]:
start = time()


colors = {
    'bitstamp': {'ask': 'rgba(255, 0, 0, 0.8)', 'bid': 'rgba(255, 0, 0, 0.3)'},
    'cex': {'ask': 'rgba(0, 255, 0, 0.8)', 'bid': 'rgba(0, 255, 0, 0.3)'},
    'exmo': {'ask': 'rgba(0, 0, 255, 0.8)', 'bid': 'rgba(0, 0, 255, 0.3)'},
    'gdax': {'ask': 'rgba(0, 0, 0, 0.8)', 'bid': 'rgba(0, 0, 0, 0.3)'},
}


traces = []
for exch in exchanges:    
    plot_data = df2[df2['bid_max_exch'] == 'bid_' + exch]
    trace = go.Scatter(x = plot_data['timestamp'],
                       y = plot_data['bid_max_val'],
                       name = 'bid_' + exch,
                       mode = 'markers',
                       marker = dict(size = 3, color = colors[exch]['bid'])
                      )
    traces.append(trace)

layout = dict(
    title = 'Max bids',
    yaxis = {'title': 'Price, USD'}
)
fig = dict(data=traces, layout=layout)
ply.iplot(fig, filename='test')


print('time: {:.3f}'.format(time() - start))

In [ ]:
start = time()


colors = {
    'bitstamp': {'ask': 'rgba(255, 0, 0, 0.8)', 'bid': 'rgba(255, 0, 0, 0.3)'},
    'cex': {'ask': 'rgba(0, 255, 0, 0.8)', 'bid': 'rgba(0, 255, 0, 0.3)'},
    'exmo': {'ask': 'rgba(0, 0, 255, 0.8)', 'bid': 'rgba(0, 0, 255, 0.3)'},
    'gdax': {'ask': 'rgba(0, 0, 0, 0.8)', 'bid': 'rgba(0, 0, 0, 0.3)'},
}


traces = []
for exch in exchanges:  
    plot_data = df2[df2['ask_min_exch'] == 'ask_' + exch]
    trace = go.Scatter(x = plot_data['timestamp'],
                       y = plot_data['ask_min_val'],
                       name = 'ask_' + exch,
                       mode = 'markers',
                       marker = dict(size = 3, color = colors[exch]['ask'])
                      )
    traces.append(trace)


layout = dict(
    title = 'Min asks',
    yaxis = {'title': 'Price, USD'}
)
fig = dict(data=traces, layout=layout)
ply.iplot(fig, filename='test')


print('time: {:.3f}'.format(time() - start))

In [ ]:
'''
i = 0
for row in df.itertuples():
    print(getattr(row, 'ask_cex'))
    i += 1
    if i == 5:
        break
''';